In [12]:
import cdsapi
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
years = np.arange(2021, 2024, 1)
title = 'ERA5_reanalysis_western_NA'

In [14]:
def get_ERA5_data(year, title):   
    # Pull ERA5 data from the CDS API client
    if not os.path.exists(f'/storage/dlhogan/data/processed_data/{title}_{year}.nc'):
        import cdsapi
        client = cdsapi.Client()
        dataset = "reanalysis-era5-pressure-levels"
        request = {
            "product_type": ["reanalysis"],
            "variable": [
                "geopotential",
                "specific_humidity",
                "temperature",
                "u_component_of_wind",
                "v_component_of_wind"
            ],
            "year": [year],
            "month": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12"
            ],
            "day": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12",
                "13", "14", "15",
                "16", "17", "18",
                "19", "20", "21",
                "22", "23", "24",
                "25", "26", "27",
                "28", "29", "30",
                "31"
            ],
            "time": [
                "00:00", "01:00", "02:00",
                "03:00", "04:00", "05:00",
                "06:00", "07:00", "08:00",
                "09:00", "10:00", "11:00",
                "12:00", "13:00", "14:00",
                "15:00", "16:00", "17:00",
                "18:00", "19:00", "20:00",
                "21:00", "22:00", "23:00"
            ],
            "pressure_level": ["500", "700"],
            "data_format": "netcdf",
            "download_format": "unarchived",
            "area": [60, -140, 20, -100]
        }
        target =  f'/storage/dlhogan/data/raw_data/{title}_{year}_v2.nc'
        client.retrieve(dataset, request, target)
        print(f'ERA5 data for {year} has been downloaded.')
        # print the approximate size of the file
        print(f'The file size is {os.path.getsize(f"/storage/dlhogan/data/raw_data/{title}_{year}_v2.nc") / 1e6} MB')
        return 
    else:
        print(f'ERA5 data for {year} has already been downloaded.')
    ds = xr.open_dataset(f'/storage/dlhogan/data/raw_data/{title}_{year}_v2.nc')
    return ds

In [16]:
for year in years:
    get_ERA5_data(year, title)

2024-11-08 12:41:19,810 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-11-08 12:41:19,811 WARNING [2024-09-26T00:00:00] Should you have not yet migrated from the old CDS system to the new CDS, please check our [informative page](https://confluence.ecmwf.int/x/uINmFw) for guidance.
2024-11-08 12:41:19,812 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-11-08 12:41:19,813 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-11-08 12:41:19,814 WARNING [2024-06-16T

KeyboardInterrupt: 

In [7]:
client = cdsapi.Client()
client.retrieve?

Signature: client.retrieve(name, request, target=None)
Docstring: <no docstring>
File:      ~/miniforge3/envs/sublime_synoptics/lib/python3.11/site-packages/cdsapi/api.py
Type:      method

In [ ]:
#! /usr/bin/env python

from mpl_toolkits.basemap import Basemap, shiftgrid, cm
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import datetime

datetime_origin = datetime.datetime(1, 1, 1, 0, 0, 0, 0)
# In this case, this is 500 mb. Can check below with d.variables["level"][:]
level = 5
time = 0
# 277830 is because this is a ~2.5 x 2.5 degree grid; 111132 meters are in about 111 km appart. This is fairly rough.
meters_per_grid = 277830

# Download the dataset from: ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc
d = Dataset("hgt.mon.mean.nc")
hgt = d.variables["hgt"][:][time, level, :, :]  # Not sure about the dims, here.
lon = d.variables["lon"][:]
# need to reverse direction of lat dimension so it's increasing.
lat = d.variables["lat"][:][::-1]
hgt = hgt[::-1, :]
hgt, lon = shiftgrid(180, hgt, lon, start = False)
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
# Lambert Azimuthal Equal Area
# Larger area:
#m = Basemap(width = 12000000, height = 12000000, resolution = "l", projection = "laea", lat_ts = 50, lat_0 = 50, lon_0 = -107.0)
# Smaller area:
#m = Basemap(width = 12000000, height = 8000000, resolution = "l", projection = "laea", lat_ts = 50, lat_0 = 50, lon_0 = -107.0)
# For north Polar Stereographic projection
m = Basemap(projection='npstere',boundinglat=10,lon_0=270,resolution='l')
nx = int((m.xmax - m.xmin)/meters_per_grid); ny = int((m.ymax - m.ymin)/meters_per_grid)
hgt = m.transform_scalar(hgt, lon, lat, nx, ny)
im = m.imshow(hgt, interpolation = "none")

m.drawcoastlines()
parallels = np.arange(-90, 90, 30)
meridians = np.arange(-180, 180, 60)
m.drawparallels(parallels, labels = [1, 0, 0, 1])
m.drawmeridians(meridians, labels = [1, 0, 0, 1])

cb = m.colorbar(im, "right", size = "5%", pad = "2%")
ax.set_title("500 mb Geopotential Height")
plt.show()